In [1]:
using HTTP, JSON, JuliaDB, DataFrames

In [2]:
"""
Queries the ARIES database of crash data maintained by the Indiana State Police. The HTML carries the following SQL query pretty printed:

SELECT * from "911880b1-d0bb-4168-a5c1-a52dcc291445" 
WHERE "PERSONTYPEDESCR" IN ('Pedal Cyclist','Pedestrian') 
AND "AGE_GRP" IN ('1-4 Years','5-14 Years','15-24 Years') 
AND "COUNTYDESCR" IN ('Marion','Boone','Hamilton','Hancock','Shelby','Johnson','Morgan','Hendricks')

This gathers data regarding child pedestrians/cyclists who live in Marion County (Indianapolis) or one of the donut counties.
"""

response = HTTP.get("https://hub.mph.in.gov/api/3/action/datastore_search_sql?sql=SELECT%20*%20from%20%22911880b1-d0bb-4168-a5c1-a52dcc291445%22%20WHERE%20%22PERSONTYPEDESCR%22%20IN%20(%27Pedal%20Cyclist%27,%27Pedestrian%27)%20AND%20%22AGE_GRP%22%20IN%20(%271-4%20Years%27,%275-14%20Years%27,%2715-24%20Years%27)%20AND%20%22COUNTYDESCR%22%20IN%20(%27Marion%27,%27Boone%27,%27Hamilton%27,%27Hancock%27,%27Shelby%27,%27Johnson%27,%27Morgan%27,%27Hendricks%27)")

HTTP.Messages.Response:
"""
HTTP/1.1 200 OK
Server: nginx/1.10.3 (Ubuntu)
Date: Fri, 30 Aug 2019 18:24:08 GMT
Content-Type: application/json;charset=utf-8
Content-Length: 921011
Connection: keep-alive
Pragma: no-cache
Cache-Control: no-cache

{"help": "https://hub.mph.in.gov/api/3/action/help_show?name=datastore_search_sql", "success": true, "result": {"records": [{"COLLISION_TIME": "07:10", "VEHUSECDE": null, "GENDERCDE": "M", "TRAFFICCNTLOPIND": "Y", "SAFETYEQUUSEDDESCR": "", "SCHOOLZONEIND": "N", "VEHLICSTATECDE": "", "TIMENOTIFIEDAMPMTXT": "PM", "PRIMARYFACTORDESCR": "PEDESTRIAN ACTION", "STATEPROPIND": "", "INVESTCOMPLETEIND": "Y", "RDWYRAMPTXT": "", "RESULTALCHTXT": "", "SPEEDLIMITTXT": "", "MANNERCOLLCDE": "12", "PRECOLLACTCDE": null, "RUMBLESTRIPIND": "N", "LIGHTCONDCDE": "3", "DISTRICT_NUM": "3", "TIMEARRIVEDTXT": "07:21", "EMGERENCY_RUN": "N", "MANNERCOLLDESCR": "Other (Explain in Narrative)", "INJNATURECDE": "12", "INCORPLIMITIND": "Y", "LONGDECIMALNMB": "0", "CONSTRUCTIND":

In [3]:
data = response.body |> String |> JSON.parse

Dict{String,Any} with 3 entries:
  "success" => true
  "help"    => "https://hub.mph.in.gov/api/3/action/help_show?name=datastore_se…
  "result"  => Dict{String,Any}("records"=>Any[Dict{String,Any}("TRAVDIRCDE"=>"…

In [4]:
tab = vcat(DataFrame.(data["result"]["records"])...) |> table

Table with 275 rows, 109 columns:
Columns:
#    colname                   type
─────────────────────────────────────────────────────
1    AGENCYORIDESCR            String
2    AGENCYORITXT              String
3    AGE_GRP                   String
4    AGGRESSIVEDRIVEIND        String
5    AXELSTXT                  Nothing
6    CITYCDE                   String
7    CITYDESCR                 String
8    COLLDTE                   String
9    COLLEVENTCDE              Nothing
10   COLLEVENTDESCR            String
11   COLLISION_DAY             String
12   COLLISION_MONTH           String
13   COLLISION_TIME            String
14   COLLISION_TIME_AM_PM      String
15   COLLISION_YEAR            String
16   CONSTRUCTIND              String
17   COUNTYCDE                 String
18   COUNTYDESCR               String
19   COUNTY_STATE              String
20   DEADNMB                   String
21   DEERNMB                   String
22   DISTRICT                  String
23   DISTRICT_NUM            